# Feature Selection Sample for WRF dataset

Demo of a feature selection procedure using PDP, ALE and Random Forest Selection

- Target observation variable : AirTemp_01HrMax

In [315]:
import os
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from darts import TimeSeries
from darts.models import NLinearModel
from darts.utils.missing_values import extract_subseries
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mae, mse, mape
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

import warnings
warnings.filterwarnings('ignore')
data_taiao = '/Dissertation/taiao_data'
# ICAO id of Wellington Airport
station_icao = 'NZAPA'

# WMO id of Wellington Airport
station_wmo = '93245'

## Seting Target

In [508]:
target = 'AirTemp_01HrMax'

## Combining all the station data in Observed dataset

In [509]:
file_paths = glob.glob(f'{data_taiao}/1minobs/{station_icao}/**/*202*.csv', recursive=True)
obs_csv = pd.concat([pd.read_csv(file, index_col='time', parse_dates=True)
                     for file in file_paths]).sort_index()

In [510]:
# Convert Celsius to Kelvin for all relevant temperature columns
obs_csv['AirTemp_01HrMax'] = obs_csv['AirTemp_01HrMax'] + 273.15
obs_csv['AirTemp_01HrMin'] = obs_csv['AirTemp_01HrMin'] + 273.15
obs_csv['AirTemp_01MnAvg'] = obs_csv['AirTemp_01MnAvg'] + 273.15
obs_csv['DewTemp_01MnAvg'] = obs_csv['DewTemp_01MnAvg'] + 273.15
obs_csv['EqpTemp_01MnAvg'] = obs_csv['EqpTemp_01MnAvg'] + 273.15

## WRF data

In [512]:
file_paths = glob.glob(f'{data_taiao}/nwp/ARWECMWFcld_single_nz4km/{station_wmo}/*202*.nc')
wrf_data = xr.open_mfdataset([file for file in file_paths],
                             engine='netcdf4',
                             concat_dim='run',
                             combine='nested',).dropna(dim='run', how='all')

### Define the Covariates

Columns to drop in wrf (more than 40% NaN): 
- 'ccld2_height', 'ccld2_depth', 'ccld1_height', 
- 'ccld1_depth', 'mcld_height', 'lcld2_height', 
- 'hcld_height', 'lcld1_height', 'tcld_height'

A few columns were dropped based on the nan values present in them

In [514]:
future_covariates_wrf = ['ccld1_amount', 'ccld2_amount', 'cldthk', 'fzl', 'hcld_amount', 
                         'itq', 'itt', 'lapprs850hPa', 'lapprs900hPa', 'laptmk900hPa', 
                         'lcld1_amount', 'lcld2_amount', 'lwdown', 'mcld_amount', 'mgust', 
                         'nhclg', 'omg500hPa', 'output_product_version', 'pblh', 'rhu500hPa', 
                         'rhu700hPa', 'rhu850hPa', 'rtb', 'rtc', 'rte', 'scape', 'sfp', 
                         'stb500hPa', 'stb500m', 'stb850hPa', 'swdown', 't2mc', 'tcld_amount', 
                         'tcldr', 'tdp10m', 'tdp700hPa', 'thw700hPa', 'thw850hPa', 'thw900hPa', 
                         'tmc10m', 'tmc500m', 'tmc700hPa', 'tmc850m', 'uuu10m', 'uuu115m', 'uuu30m', 
                         'uuu45m', 'uuu500hPa', 'uuu500m', 'uuu60m', 'uuu700hPa', 'uuu75m', 
                         'uuu850hPa', 'uuu90m', 'vvv10m', 'vvv115m', 'vvv30m', 'vvv45m', 'vvv500hPa', 
                         'vvv500m', 'vvv60m', 'vvv700hPa', 'vvv75m', 'vvv850hPa', 'vvv90m', 'wdr500m', 
                         'wdr700hPa', 'wdr850hPa', 'wspk10m', 'wspk115m', 'wspk200m', 'wspk30m', 
                         'wspk45m', 'wspk500hPa', 'wspk500m', 'wspk60m', 'wspk75m', 'wspk850hPa', 
                         'wspk90m', 'www500hPa', 'www700hPa', 'www900hPa']

### Merge obs and NWP data

In [ ]:
# Add the valid_time of the NWP forecast, to join with the observation time
wrf_data_df = (wrf_data.to_dataframe().reset_index()
          .assign(valid_time = lambda x : x.run + x.prognosis_period).set_index('valid_time'))
wrf_data_df.shape

In [516]:
# List of columns in Celsius that need to be converted to Kelvin
celsius_columns = ['TMaxDaily', 'TMinDaily','t2_24h_max','t2_24h_min','t2m', 't850p', 'tdwpt', 'tmax', 'tmin']

# Convert each column from Celsius to Kelvin by adding 273.15
for col in celsius_columns:
    if col in wrf_data_df.columns:  # Ensure the column exists in the DataFrame
        wrf_data_df[col] = wrf_data_df[col] + 273.15

In [ ]:
joined_df = obs_csv[[target]].join(wrf_data_df[['run', 'prognosis_period'] + future_covariates_wrf], how='inner').dropna()


In [ ]:
deduped_df = joined_df.reset_index().sort_values(['time', 'run']).drop_duplicates(subset='time', keep='last').set_index('time')
deduped_df = deduped_df.drop(['run', 'prognosis_period'], axis=1)


In [519]:
train_df = deduped_df.loc[deduped_df.index.year < 2022]
valid_df = deduped_df.loc[deduped_df.index.year == 2022]
test_df = deduped_df.loc[deduped_df.index.year == 2023]

In [524]:
# lets try to forecast 12h ahead based on the previous 12h
input_chunk_length=2
output_chunk_length=2

In [ ]:
import numpy as np
from darts.metrics import mae, mse, mape

# Test metrics with last_points_only=True
metrics = model.backtest(
    series=test_target,
    future_covariates=test_future_cov,
    forecast_horizon=output_chunk_length,
    metric=[mae, mse, mape],
    last_points_only=True,  # Only consider the last point of each forecast
    retrain=False
)

# Calculate the average metrics across all iterations
average_metrics = np.mean(metrics, axis=0)
print("Average MAE, MSE, MAPE:", average_metrics)


## Feature Selection Using Random Forest

#### Preprocess the Dataset

In [529]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train = train_df.drop(columns=[target])
X_valid = valid_df.drop(columns=[target])
y_train = train_df[target]
y_valid = valid_df[target]

#### Train a Random Forest Model

In [530]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from PyALE import ale

# Train the RandomForestRegressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model on validation set
accuracy_before = rf.score(X_valid, y_valid)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')


#### Extracting feature importances and ranking them

In [531]:
# Extract feature importances
importances = rf.feature_importances_
feature_names = X_train.columns
feature_importance_df = pd.DataFrame({'Feature_RF': feature_names, 'Importance_RF': importances})

# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance_RF', ascending=False)
feature_importance_df=feature_importance_df.head(20)

top_features = feature_importance_df['Feature_RF'][:20].values
X_train_selected = X_train[top_features]
X_valid_selected = X_valid[top_features]


#### Re-train the model with the selected top features

In [ ]:
# Re-train the model with the selected top features
rf2 = RandomForestRegressor(n_estimators=100, random_state=42)
rf2.fit(X_train_selected, y_train)  # Ensure to use the selected features here

# Evaluate the model on validation set
accuracy_after = rf2.score(X_valid_selected, y_valid)
print(f'Accuracy after feature selection: {accuracy_after:.3f}')
y_predRF = rf2.predict(X_valid_selected)


### ALE PLOT FOR Randomforest

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Number of features
num_features = len(top_features)

# Set up a grid of subplots (e.g., 4 rows and 5 columns for 20 features)
rows, cols = 4, 5
fig, axes = plt.subplots(rows, cols, figsize=(15, 10))
axes = axes.flatten()  # Flatten the 2D array of axes to 1D for easier indexing

# Dictionary to store ALE importance values
ale_forest_importance = {}

# Iterate through each feature and plot ALE
for idx, feature in enumerate(top_features):
    ale_eff = ale(
        X=X_train_selected,  # The test data as a pandas DataFrame
        model=rf2,  # The model
        feature=[feature],  # Feature for which ALE is computed
        grid_size=10,  # Adjust the grid size as needed
        C=0.95  # 95% confidence interval
    )
    
    # Get the ALE effect values
    ale_effects = ale_eff['eff']
    feature_values = ale_eff.index  
    # Plot the ALE effect in the appropriate subplot
    axes[idx].plot(feature_values, ale_effects, label=f'ALE of {feature}')
    axes[idx].set_title(f'ALE for {feature}')
    axes[idx].set_xlabel(feature)
    axes[idx].set_ylabel('ALE Effect')
    
    # Optional: Customize appearance (like grid, labels, etc.)
    axes[idx].grid(True)
    
    # Calculate the mean absolute ALE effect for the feature
    mean_abs_ale = np.mean(np.abs(ale_effects))
    ale_forest_importance[feature] = mean_abs_ale

# Remove any unused subplots
for ax in axes[num_features:]:
    fig.delaxes(ax)

# Adjust layout so the plots don't overlap
fig.tight_layout()

# Show the figure (optional)
plt.show()


In [534]:
# Convert the feature importance to a DataFrame and sort by importance
ale_forest_importance_df = pd.DataFrame(list(ale_forest_importance.items()), columns=['Feature_AleRF', 'Importance_AleRF'])
ale_forest_importance_df = ale_forest_importance_df.sort_values(by='Importance_AleRF', ascending=False)
csv_df= pd.concat([feature_importance_df.reset_index(drop=True),ale_forest_importance_df.reset_index(drop=True)],axis=1)

## Training MLP with StandardScaler

In [535]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train_selected)
X_valid_scaled = scaler.transform(X_valid_selected)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_selected.columns)
# Initialize the MLP Regressor
mlp = MLPRegressor(hidden_layer_sizes=(120),  # You can adjust these values
                   activation='tanh', 
                   solver='sgd',
                   max_iter=1000, 
                   random_state=42
                   )

# Train the model
mlp.fit(X_train_scaled, y_train)

# Predict on validation set
y_pred = mlp.predict(X_valid_scaled)


### ALE PLOT FOR MLP

In [ ]:
# Set up a grid of subplots (e.g., 4 rows and 5 columns for 20 features)
rows, cols = 4, 5
fig, axes_mlp = plt.subplots(rows, cols, figsize=(15, 10))
axes_mlp = axes_mlp.flatten()  # Flatten the 2D array of axes to 1D for easier indexing
mlp_importance = {}
# Iterate through each feature and calculate ALE
for idx, feature in enumerate(top_features):
    print(f"Calculating ALE for {feature}...")
    
    # Calculate the ALE for the current feature
    ale_eff = ale(
        X=X_train_scaled_df,           # The scaled training data
        model=mlp,              # The trained MLP Regressor model
        feature=[feature],      # The current feature to analyze
        grid_size=10,           # You can adjust the grid size as needed
        C=0.95                  # 95% confidence interval
    )
    
    # Get the ALE effect values (typically stored in the 'eff' column of the result)
    ale_effects = ale_eff['eff']
    feature_values = ale_eff.index  
    # Plot the ALE effect in the appropriate subplot
    axes_mlp[idx].plot(feature_values, ale_effects, label=f'ALE of {feature}')
    axes_mlp[idx].set_title(f'ALE for {feature}')
    axes_mlp[idx].set_xlabel(feature)
    axes_mlp[idx].set_ylabel('ALE Effect')
    axes_mlp[idx].grid(True)
    # Calculate the mean absolute ALE effect for the feature
    mean_abs_ale = np.mean(np.abs(ale_effects))
    
    # Store the feature and its importance score
    mlp_importance[feature] = mean_abs_ale
# Remove any unused subplots
for ax in axes_mlp[num_features:]:
    fig.delaxes(ax)

# Adjust layout so the plots don't overlap
fig.tight_layout()

# Show the figure (optional)
plt.show()


In [537]:
# Convert the feature importance dictionary to a DataFrame and sort it
mlp_ale_importance_df = pd.DataFrame(list(mlp_importance.items()), columns=['Feature_Ale_MLP', 'Importance_Ale_MLP'])
mlp_ale_importance_df = mlp_ale_importance_df.sort_values(by='Importance_Ale_MLP', ascending=False)
mlp_ale_importance_df['Importance_Ale_MLP'] = mlp_ale_importance_df['Importance_Ale_MLP'].apply(lambda x: f'{x:.6f}')

In [538]:
csv_df= pd.concat([mlp_ale_importance_df.reset_index(drop=True),csv_df.reset_index(drop=True)],axis=1)

### PDP plot for mlp and storing the ranking(stadard deviation wise)

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Number of columns in the grid
n_cols = 3  
n_rows = (len(top_features) + n_cols - 1) // n_cols  # Calculate rows based on the number of features and columns

# Create a figure with subplots (adjust size accordingly)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  
axes = axes.flatten()  # Flatten axes to iterate easily

# Create a dictionary to store the PDP importance scores
mlp_pdp_stddev = {}

# Iterate through each feature and calculate PDP
for idx, feature in enumerate(top_features):  
    print(f"Calculating PDP for {feature}...")
    
    # Get the feature index in X_train_selected (since PDP works on indices, not names)
    feature_idx = X_train_scaled_df.columns.get_loc(feature)
    
    # Calculate the PDP for the current feature and plot it on the respective subplot
    display = PartialDependenceDisplay.from_estimator(
        mlp, 
        X=X_train_scaled, 
        features=[feature_idx], 
        ax=axes[idx],  # Use the appropriate subplot
        grid_resolution=100
    )
    
    # Set the title for each subplot
    axes[idx].set_title(f"PDP for {feature}")
    
    # Extract the partial dependence values from the display object
    pdp_effects = display.pd_results[0]['average'][0]
    
    # Calculate the standard deviation of PDP effects for the feature
    stddev_pdp = np.std(pdp_effects)
    
    # Store the feature and its standard deviation score
    mlp_pdp_stddev[feature] = stddev_pdp

# Hide unused subplots if there are any
for i in range(len(top_features), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figure with all PDP plots
plt.savefig('pdp_MLP_plots.png', dpi=300)

# Show the combined plot
plt.show()

# Convert the PDP standard deviation dictionary to a DataFrame and sort it
mlp_pdp_stddev_df = pd.DataFrame(list(mlp_pdp_stddev.items()), columns=['Feature_PDP_MLP', 'StdDev_PDP_MLP'])
mlp_pdp_stddev_df = mlp_pdp_stddev_df.sort_values(by='StdDev_PDP_MLP', ascending=False)
mlp_pdp_stddev_df['StdDev_PDP_MLP'] = mlp_pdp_stddev_df['StdDev_PDP_MLP'].apply(lambda x: f'{x:.6f}')

# Display the ranked features based on PDP standard deviation
print("Ranked feature importance based on PDP Standard Deviation:")
print(mlp_pdp_stddev_df)


In [540]:
csv_df= pd.concat([mlp_pdp_stddev_df.reset_index(drop=True),csv_df.reset_index(drop=True)],axis=1)

### PDP plot for RF

In [ ]:
from sklearn.inspection import partial_dependence
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt
import numpy as np

# Number of columns in the grid
n_cols = 3  
n_rows = (len(top_features) + n_cols - 1) // n_cols  # Calculate rows based on the number of features and columns

# Create a figure with subplots (adjust size accordingly)
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))  
axes = axes.flatten()  # Flatten axes to iterate easily

# Create a dictionary to store the PDP standard deviation scores
rf_pdp_stddev = {}

# Iterate through each feature and calculate PDP
for idx, feature in enumerate(top_features):  
    print(f"Calculating PDP for {feature}...")
    
    # Get the feature index in X_train_selected (since PDP works on indices, not names)
    feature_idx = X_train_scaled_df.columns.get_loc(feature)
    
    # Calculate the PDP for the current feature and plot it on the respective subplot
    display = PartialDependenceDisplay.from_estimator(
        rf2, 
        X=X_train_selected, 
        features=[feature_idx], 
        ax=axes[idx],  # Use the appropriate subplot
        grid_resolution=100
    )

    # Set the title for each subplot
    axes[idx].set_title(f"PDP for {feature}")

    # Extract the partial dependence values from the display object
    pdp_effects = display.pd_results[0]['average'][0]
    
    # Calculate the standard deviation of PDP effects for the feature
    stddev_pdp = np.std(pdp_effects)
    
    # Store the feature and its standard deviation score
    rf_pdp_stddev[feature] = stddev_pdp

# Hide unused subplots if there are any
for i in range(len(top_features), len(axes)):
    fig.delaxes(axes[i])

# Adjust layout to prevent overlap
plt.tight_layout()

# Save the figure with all PDP plots
plt.savefig('pdp_RF_plots.png', dpi=300)

# Show the combined plot
plt.show()

# Convert the PDP standard deviation dictionary to a DataFrame and sort it
rf_pdp_stddev_df = pd.DataFrame(list(rf_pdp_stddev.items()), columns=['Feature_PDP_RF', 'StdDev_PDP_RF'])
rf_pdp_stddev_df = rf_pdp_stddev_df.sort_values(by='StdDev_PDP_RF', ascending=False)
rf_pdp_stddev_df['StdDev_PDP_RF'] = rf_pdp_stddev_df['StdDev_PDP_RF'].apply(lambda x: f'{x:.6f}')

# Display the ranked features based on PDP standard deviation
print("Ranked feature importance based on PDP Standard Deviation:")
print(rf_pdp_stddev_df)


In [542]:
csv_df= pd.concat([rf_pdp_stddev_df.reset_index(drop=True),csv_df.reset_index(drop=True)],axis=1)

### Calculating MLP and RFs' MAE, MSE and R2 error

In [ ]:
# Calculate regression metrics for Random Forest
mse_rf = mean_squared_error(y_valid, y_predRF)
mae_rf = mean_absolute_error(y_valid, y_predRF)
r2_rf = r2_score(y_valid, y_predRF)
print(f'Model: RF')
print(f'Mean Squared Error (MSE): {mse_rf:.4f}')
print(f'Mean Absolute Error (MAE): {mae_rf:.4f}')
print(f'R-squared (R2): {r2_rf:.4f}')

# Calculate regression metrics for MLP
mse_mlp = mean_squared_error(y_valid, y_pred)
mae_mlp = mean_absolute_error(y_valid, y_pred)
r2_mlp = r2_score(y_valid, y_pred)
print(f'Model: MLP')
print(f'Mean Squared Error (MSE): {mse_mlp:.4f}')
print(f'Mean Absolute Error (MAE): {mae_mlp:.4f}')
print(f'R-squared (R2): {r2_mlp:.4f}')

### Store metrics in csv

In [ ]:
# Define the metrics for both RF and MLP
metrics_data = {
    'Model': ['RF', 'MLP'],
    'MSE': [mse_rf, mse_mlp],  # mse_rf and mse_mlp from the respective models
    'MAE': [mae_rf, mae_mlp],  # mae_rf and mae_mlp from the respective models
    'R2': [r2_rf, r2_mlp]      # r2_rf and r2_mlp from the respective models
}

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics_data)

# Concatenate the metrics to the feature importance data
# Reset index to avoid index alignment issues
csv_df = pd.concat([csv_df.reset_index(drop=True), metrics_df.reset_index(drop=True)], axis=1)

# Save the updated DataFrame to a CSV file
csv_df.to_csv('Feature_ECMWF_Ensemble_with_metrics.csv', index=False)

print("Metrics and feature importances saved to 'Feature_ECMWF_Ensemble_with_metrics.csv'")
